In [52]:
from typing import Dict, Set, Tuple, List
from heapq import heappop, heappush

RAW = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""


RAW_HUGE = """11637517422274862853338597396444961841755517295286
13813736722492484783351359589446246169155735727126
21365113283247622439435873354154698446526571955763
36949315694715142671582625378269373648937148475914
74634171118574528222968563933317967414442817852555
13191281372421239248353234135946434524615754563572
13599124212461123532357223464346833457545794456865
31254216394236532741534764385264587549637569865174
12931385212314249632342535174345364628545647573965
23119445813422155692453326671356443778246755488935
22748628533385973964449618417555172952866628316397
24924847833513595894462461691557357271266846838237
32476224394358733541546984465265719557637682166874
47151426715826253782693736489371484759148259586125
85745282229685639333179674144428178525553928963666
24212392483532341359464345246157545635726865674683
24611235323572234643468334575457944568656815567976
42365327415347643852645875496375698651748671976285
23142496323425351743453646285456475739656758684176
34221556924533266713564437782467554889357866599146
33859739644496184175551729528666283163977739427418
35135958944624616915573572712668468382377957949348
43587335415469844652657195576376821668748793277985
58262537826937364893714847591482595861259361697236
96856393331796741444281785255539289636664139174777
35323413594643452461575456357268656746837976785794
35722346434683345754579445686568155679767926678187
53476438526458754963756986517486719762859782187396
34253517434536462854564757396567586841767869795287
45332667135644377824675548893578665991468977611257
44961841755517295286662831639777394274188841538529
46246169155735727126684683823779579493488168151459
54698446526571955763768216687487932779859814388196
69373648937148475914825958612593616972361472718347
17967414442817852555392896366641391747775241285888
46434524615754563572686567468379767857948187896815
46833457545794456865681556797679266781878137789298
64587549637569865174867197628597821873961893298417
45364628545647573965675868417678697952878971816398
56443778246755488935786659914689776112579188722368
55172952866628316397773942741888415385299952649631
57357271266846838237795794934881681514599279262561
65719557637682166874879327798598143881961925499217
71484759148259586125936169723614727183472583829458
28178525553928963666413917477752412858886352396999
57545635726865674683797678579481878968159298917926
57944568656815567976792667818781377892989248891319
75698651748671976285978218739618932984172914319528
56475739656758684176786979528789718163989182927419
67554889357866599146897761125791887223681299833479"""


XY = Tuple[int, int]
Grid = Dict[XY, int]

class Route:
    
    def __init__(self, text):
        self.grid = self.parse(text=text)
        self.xmax = max(x for x, y in self.grid)
        self.ymax = max(y for x, y in self.grid)
        self.lines = [[int(num) for num in line]
                      for line in text.splitlines()]
        
    
    def parse(self, text)->Grid:
        lines = text.splitlines()
        grid = {}
        for y in range(len(lines)):
            for x in range(len(lines[0])):
                grid[(x,y)] = int(lines[y][x])
        return grid
    
    def find_lowest_path(self)->int:
        """
        Finds the lowest path tp the bottom right corner
        of the maze
        """
        
        visited = {(0,0)}
        tree = [] 
        # (cost, x, y)
        heappush(tree, (0, 0, 0))
        while tree:
            #print(tree)
            cost, x, y = heappop(tree)
            #print(cost, (x, y))
            if x == self.xmax and y == self.ymax:
                return cost
            for x_, y_  in self.neighbours((x, y), visited):
                heappush(tree, (cost + self.grid[(x_, y_)], x_, y_))
                visited.add((x_, y_))
        raise Exception('No lowest path found')
    

    
    def neighbours(self, current:XY, visited:Set[XY]):
        """
        (...) (0,-1) (...)
        (-1,0)       (1,0)
        (...) (0, 1) (...)
        """
        x, y = current
        for dx, dy in [(0,-1), (-1,0), (1,0), (0,1)]:
            new_loc = x+dx, y+dy
            if  new_loc not in visited and 0<=x+dx<=self.xmax and 0<=y+dy<=self.ymax:
                yield x+dx, y+dy
                
                
    @staticmethod
    def lines_to_str(lines:List[List[int]]):
        """
        Converts  the growth new grid
        to a str format has the problem poses
        """
        
        new_grid = []
        for line in lines:
            row = "".join(str(value)
                   for value in line)
            new_grid.append(row)
        return "\n".join(new_grid)
    
    def shrink(self, x: int) -> int:
        while x > 9:
            x -= 9
        return x
    

    def grow(self) -> List[List[int]]:
    
        new_grid = [[0 for _ in range((self.xmax + 1) * 5)] for _ in range((self.ymax + 1) * 5)]
        
        # 5 x 5 tiles expansion of grid plus
        # increment of 1 per tile create left and right
        for i in range(5):
            for j in range(5):
                for r in range(self.ymax+1):
                    for c in range(self.xmax+1):
                        new_value = self.shrink(self.lines[r][c] + i + j)
                        #print(i, j, r, c)
                        new_grid[r + i * (self.ymax+1)][c + j * (self.xmax+1)] = new_value
                        # print(i, j, r, c, new_value)
        return new_grid


    

R = Route(text=RAW)
assert R.find_lowest_path() == 40
GROWTH_TEXT = Route.lines_to_str(R.grow())
assert RAW_HUGE == GROWTH_TEXT
R2 = Route(text=GROWTH_TEXT)
assert R2.find_lowest_path() == 315

with open('inputs/day15.txt') as f:
    text = f.read()
    r = Route(text=text)
    print('p1', r.find_lowest_path())
    growth_text = Route.lines_to_str(r.grow())
    r2 = Route(text=growth_text)
    print('p2', r2.find_lowest_path())

p1 487
p2 2821


In [ ]:
def shrink(x:int)->int:
    while x > 9:
        x -= 9
    return x
        
def parse(text:str)-> List[List[int]]:
    return [[int(num) for num in line]
             for line in text.splitlines()]

xmax = 9
ymax = 9
new_grid = [[0 for _ in range((xmax + 1) * 5)] for _ in range((ymax + 1) * 5)]